In [ ]:
!git clone https://github.com/NVIDIA/Megatron-LM.git
%cd Megatron-LM/

In [ ]:
!pip install transformers datasets huggingface_hub

In [ ]:
import os
import json
from huggingface_hub import snapshot_download
from datasets import load_dataset
from megatron.core.tokenizers import MegatronTokenizer

In [ ]:
os.makedirs("data", exist_ok=True)
ds = load_dataset("roneneldan/TinyStories", split="train")
dataset = ds.select(range(10000))
with open("data/tinystories_train.jsonl", "w", encoding="utf-8") as f:
    for row in dataset:
        f.write(json.dumps({"text": row["text"]}) + "\n")

In [ ]:
tok_dir = snapshot_download(
    repo_id="gpt2",
    allow_patterns=[
        "tokenizer.json", "vocab.json", "merges.txt",
        "tokenizer_config.json", "special_tokens_map.json", "*.model"
    ]
)
MegatronTokenizer.write_metadata(
    tokenizer_path=tok_dir,
    tokenizer_library="huggingface",
    metadata_path=os.path.join("data/", "tokenizer_metadata.json")
)

In [ ]:
!python tools/preprocess_data.py \
  --input data/tinystories_train.jsonl \
  --json-keys text \
  --output-prefix data/tinystories_meg \
  --tokenizer-type HuggingFaceTokenizer \
  --tokenizer-model gpt2 \
  --append-eod \
  --workers 8